<a href="https://colab.research.google.com/github/Asadali016/VGG16/blob/main/Phase_1_%26_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/Asadali016/VGG16.git

In [2]:
#Code for Extracting Features using VGG16 'Start'
from collections import Counter
import cv2
import os
import glob
import skimage
import numpy as np
import pandas as pd
import seaborn as sn
from tqdm import tqdm
from PIL import Image
from os import listdir
import matplotlib.pyplot as plt
from skimage.transform import resize
from collections import Counter

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import AUC
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.applications.vgg16 import VGG16 # VGG16
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Flatten, Activation, GlobalAveragePooling2D,Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [3]:
images=[]
labels=[]
feature_dictionary = {
    'label': tf.io.FixedLenFeature([], tf.int64),
    'label_normal': tf.io.FixedLenFeature([], tf.int64),
    'image': tf.io.FixedLenFeature([], tf.string)
    }

In [4]:
def _parse_function(example, feature_dictionary=feature_dictionary):
    parsed_example = tf.io.parse_example(example, feature_dictionary)
    return parsed_example

def read_data(filename):
    full_dataset = tf.data.TFRecordDataset(filename,num_parallel_reads=tf.data.experimental.AUTOTUNE)
    full_dataset = full_dataset.shuffle(buffer_size=31000)
    full_dataset = full_dataset.cache()
    print("Size of Training Dataset: ", len(list(full_dataset)))
    
    feature_dictionary = {
    'label': tf.io.FixedLenFeature([], tf.int64),
    'label_normal': tf.io.FixedLenFeature([], tf.int64),
    'image': tf.io.FixedLenFeature([], tf.string)
    }   

    full_dataset = full_dataset.map(_parse_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    print(full_dataset)
    for image_features in full_dataset:
        image = image_features['image'].numpy()
        image = tf.io.decode_raw(image_features['image'], tf.uint8)
        image = tf.reshape(image, [299, 299])        
        image=image.numpy()
        image=cv2.resize(image,(100,100))
        image=cv2.merge([image,image,image])        
        #plt.imshow(image)
        images.append(image)
        labels.append(image_features['label_normal'].numpy())


In [ ]:
filenames=['/content/drive/MyDrive/Fiverr_WORK/training10_1/training10_1.tfrecords']
for filenames in filenames:
    read_data(filenames)   
print(len(images))
print(len(labels))

In [6]:
X=np.array(images)
y=np.array(labels)
print(len(X))
print(len(y))
#x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2021,shuffle=True)

11177
11177


In [ ]:
plt.imshow(X[0])

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Conv2D, MaxPool2D, Flatten
from keras import optimizers
from keras import losses
from sklearn import metrics

print(X[0].shape)

In [ ]:
rows, cols,color = X[0].shape
print(X[0].shape)

base_model = VGG16(input_shape=(100,100,3), weights='imagenet', include_top=False)
x = base_model.output
x = Dropout(0.2)(x)
predictions = Flatten()(x)
model_feat = Model(inputs=base_model.input,outputs=predictions)

features = model_feat.predict(X)


In [ ]:
import pandas as pd 
pd.DataFrame(features).to_csv('/content/drive/MyDrive/Fiverr_WORK/features.csv')

In [ ]:
import pandas as pd  
features1= pd.read_csv('/content/drive/MyDrive/Fiverr_WORK/new_data.csv')
print(len(features1))
print(len(y))
features1['label'] = y
pd.DataFrame(features1).to_csv('/content/drive/MyDrive/Fiverr_WORK/new_Data.csv')
#Code for Extracting Features using VGG16 'End'

In [59]:
#Code for Training SVM 'Start'
import pandas as pd
dataset=pd.read_csv('/content/drive/MyDrive/Fiverr_WORK/New_data.csv')

In [60]:
X=dataset.iloc[:,:-1].values
Y=dataset.iloc[:,-1].values

In [61]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.25, random_state=0)

In [62]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [ ]:
from sklearn.svm import SVC
classifier=SVC(kernel='linear',random_state=0)
classifier.fit(X_train,Y_train)

In [ ]:
Y_pred=classifier.predict(X_test)
from sklearn.metrics import accuracy_score, confusion_matrix
cm=confusion_matrix(Y_test,Y_pred)
print(cm)

In [ ]:
ac=accuracy_score(Y_test,Y_pred)
print(ac)
#Code for SVM 'End'

In [77]:
#Code for NADE 'Start'
import torch
from torch import distributions
from torch import nn


class NADE():
    def __init__(self, input_dim, hidden_dim):
        """Initializes a new NADE instance.
        Args:
            input_dim: The dimension of the input.
            hidden_dim: The dimmension of the hidden layer. NADE only supports one
                hidden layer.
        """
        super().__init__()
        self._input_dim = input_dim

        # fmt: off
        self._in_W = nn.Parameter(torch.zeros(hidden_dim, self._input_dim))
        self._in_b = nn.Parameter(torch.zeros(hidden_dim,))
        self._h_W = nn.Parameter(torch.zeros(self._input_dim, hidden_dim))
        self._h_b = nn.Parameter(torch.zeros(self._input_dim,))
        # fmt: on
        nn.init.kaiming_normal_(self._in_W)
        nn.init.kaiming_normal_(self._h_W)

    def _forward(self, x):
        """Computes the forward pass and samples a new output.
        Returns:
            (p_hat, x_hat) where p_hat is the probability distribution over dimensions
            and x_hat is sampled from p_hat.
        """
        # If the input is an image, flatten it during the forward pass.
        original_shape = x.shape
        if len(x.shape) > 2:
            x = x.view(original_shape[0], -1)

        p_hat, x_hat = [], []
        batch_size = 1 if x is None else x.shape[0]
        # Only the bias is used to compute the first hidden unit so we must replicate it
        # to account for the batch size.
        a = self._in_b.expand(batch_size, -1)
        for i in range(self._input_dim):
            h = torch.relu(a)
            p_i = torch.sigmoid(h @ self._h_W[i : i + 1, :].t() + self._h_b[i : i + 1])
            p_hat.append(p_i)

            # Sample 'x' at dimension 'i' if it is not given.
            x_i = x[:, i : i + 1]
            x_i = torch.where(x_i < 0, distributions.Bernoulli(probs=p_i).sample(), x_i)
            x_hat.append(x_i)

            # We do not need to add self._in_b[i:i+1] when computing the other hidden
            # units since it was already added when computing the first hidden unit.
            a = a + x_i @ self._in_W[:, i : i + 1].t()
        if x_hat:
            return (
                torch.cat(p_hat, dim=1).view(original_shape),
                torch.cat(x_hat, dim=1).view(original_shape),
            )
        return []

    def classifier():
      from sklearn.naive_bayes import GaussianNB
      c=GaussianNB()
      return c

    def forward(self, x):
        """Computes the forward pass.
        Args:
            x: Either a tensor of vectors with shape (n, input_dim) or images with shape
                (n, 1, h, w) where h * w = input_dim.
        Returns:
            The result of the forward pass.
        """
        return self._forward(x)[0]

    def sample(self, n_samples=None, conditioned_on=None):
        """See the base class."""
        with torch.no_grad():
            conditioned_on = self._get_conditioned_on(n_samples, conditioned_on)
            return self._forward(conditioned_on)[1]


def reproduce(
    n_epochs=50,
    batch_size=512,
    log_dir="/tmp/run",
    n_gpus=1,
    device_id=0,
    debug_loader=None,
):
    from torch import optim
    from torch.nn import functional as F

    from pytorch_generative import datasets
    from pytorch_generative import models
    from pytorch_generative import trainer

    train_loader, test_loader = debug_loader, debug_loader
    if train_loader is None:
        train_loader, test_loader = datasets.get_mnist_loaders(
            batch_size, dynamically_binarize=True
        )

    model = models.NADE(input_dim=784, hidden_dim=500)
    optimizer = optim.Adam(model.parameters())

    def loss_fn(x, _, preds):
        batch_size = x.shape[0]
        x, preds = x.view((batch_size, -1)), preds.view((batch_size, -1))
        loss = F.binary_cross_entropy_with_logits(preds, x, reduction="none")
        return loss.sum(dim=1).mean()

    model_trainer = trainer.Trainer(
        model=model,
        loss_fn=loss_fn,
        optimizer=optimizer,
        train_loader=train_loader,
        eval_loader=test_loader,
        log_dir=log_dir,
        n_gpus=n_gpus,
        device_id=device_id,
    )
    model_trainer.interleaved_train_and_eval(n_epochs)


In [72]:
import pandas as pd
dataset=pd.read_csv('/content/drive/MyDrive/Fiverr_WORK/positive_multi.csv')
Data_multi=pd.read_csv('/content/drive/MyDrive/Fiverr_WORK/data.csv')

In [73]:
X_M=dataset.iloc[:,:-1].values
Y_multi=dataset.iloc[:,-1].values

In [80]:
from sklearn.model_selection import train_test_split
X_train_M,X_test_M,Y_train_M,Y_test_M=train_test_split(X_M,Y_multi,test_size=0.25, random_state=0)

In [81]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train_M=sc.fit_transform(X_train_M)
X_test_M=sc.transform(X_test_M)

In [ ]:
Classifier=NADE.classifier()
Classifier.fit(X_train_M,Y_train_M)

In [ ]:
Y_pred_M=Classifier.predict(X_test_M)
from sklearn.metrics import accuracy_score, confusion_matrix
cm=confusion_matrix(Y_test_M,Y_pred_M)
print(cm)

In [ ]:
ac=accuracy_score(Y_test_M,Y_pred_M)
print(ac)
#Code for NADE 'End'

In [ ]:
#Code for K-NN Classifier
from sklearn.neighbors import KNeighborsClassifier
CLassifier=KNeighborsClassifier(n_neighbors=50,metric='minkowski',p=5) 
CLassifier.fit(X_train_M,Y_train_M)
y_pred_1=CLassifier.predict(X_test_M)
Cm=confusion_matrix(Y_test_M,y_pred_1)
print(Cm)
Ac=accuracy_score(Y_test_M,y_pred_1)
print(Ac)


In [103]:
#Code for Testing
actual=Y_test[10]
actual_M=Y_test_M[10]
print('Original Values',actual)
print('Original Multi_class Value',actual_M)
input=X_test[10,:]
input_M=X_test_M[10,:]
input=input.reshape(1,-1)
input_M=input_M.reshape(1,-1)
check=classifier.predict(input)
if(check==0):
  print('Output=Negative')
else :
  print('Output=Positive')
  check1=Classifier.predict(input_M)
  print('Predicted by NADE',check1)
  check2=CLassifier.predict(input_M)
  print('Predicted by K-NN',check2)





Original Values 0
Original Multi_class Value 3
Output=Negative
